
# <center>  Tutorial 20 Nov </center>
# <center>  Structure at Macro, Meso and Nano Scale </center>

## <center>  Fabian IJpelaar </center>
### <center>f.ijpelaar@rug.nl </center>
---


# My current project: Building a library for fast simulations of Ising models
- ## We hope that Ising models can provide a bridge between machine learning and material (neuromorphic) computing
___

## Topics:
### - Introduction into Monte Carlo Simulations
### - Demonstrations of the Ising Model
### - Ising Models in Machine Learning: The Boltzmann Machine

# Monte Carlo Simulations
- Monte Carlo methods are a class of computational algorithms where we use use random samples to obtain numerical results
- I.e. we might want to estimate averages of some function $\langle f(x) \rangle = \sum_{x} f(x) p(x)$
- Or we might want to generate examples from a probability distirbution

# Monte Carlo
## Well Known Example: Estimate $\pi$
<center><img src="Circle.png" width="400" align="center"/></center>

### Probability to be in circle: $p_{in} = \frac {A_{Circle}} {A_{Total}} = \frac \pi 4$

In [1]:
include("MakiePi.jl")
pi_est = estimatePi()

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Observable(4.0)
    0 => map((::var"#11#16")(est) @ Main ~/Library/Mobile Documents/com~apple~CloudDocs/Documents/PhD/JuliaProjects/InteractiveIsing.jl/examples/Tutorial/MakiePi.jl:39)


# Markov Chain Monte Carlo
### - Generating states out of the whole state space with appropriate probability might be hard
### - Solution: Limit generation of states to states that are in the neighborhood, 
### - And define transition probabilities $P(newstate|oldstate)$

### We create a random walk through state space
<center><img src="Markov Chain.png" width="300" align="center"/></center>

____


# Metropolis Algorithm for Ising Models:
<img src="Metropolis.png" width="400" align="center"/>

### For each spin in the lattice:
1. Randomly select a spin in the lattice.

2. ΔE: Calculate the energy change if that spin would be flipped.

3. If ΔE <= 0 (energy is lowered):
       
    - Flip the spin (accept the new state).
    
    - Go to 1.

4. Else (ΔE > 0):
       
    - Calculate the acceptance factor, $e^{-\betaΔE}$ i.e. the "Boltzmann Factor"
            
    - r: Generate a random number between 0 and 1.
            
    - If $r < e^{-\betaΔE}$:
        - Flip the spin (accept the new state despite higher energy).
        
    - Else: Do not flip the spin (reject the new state).
5. Go to 1.

##### If we choose the acceptance factor to be the Boltzmann factor, over long enough time the distribution will be the same as the Boltzmann Distribution
##### $$ P(\sigma_1,\sigma_2,...) = \frac 1 Z e^{-\beta H} $$
____


# Initializing the graph and starting the simulation
##### $H = -\sum_{ij} W_{ij}\sigma_i\sigma_j -\sum_{i} b_i \sigma_i$

In [1]:
using InteractiveIsing

# Create the graph
ig = IsingGraph(200,200, type = Discrete)
# 
simulate(ig, gui = false, overwrite = true)

# Generate the weights for the IsingModel
wg = @WG "dr -> dr == 1 ? 1 : 0" NN=1
genAdj!(ig[1], wg)

# Create a screen to display the graph
layerwindow = LayerWindow(ig[1]);

# Demonstation of the Second Order Phase Transition in the Ising Model

In [26]:
w = createAnalysisWindow(ig, MT_panel, tstep = 0.01);

# Demonstration of Hysteresis
- ### Q: Why do we get hysteresis here?

In [27]:
w = createAnalysisWindow(ig, MB_panel);

# Demonstration of Divergence of Isothermal Susceptibility

- ### We can get the Isothermal Susceptibility from the fluctuations in samples we take of the magnetization
- ### $\chi_T = \left( \frac{\partial M}{\partial H} \right)_T = \beta ( \langle M^2 \rangle - \langle |M| \rangle ^2)$

In [28]:
w = createAnalysisWindow(ig, χₘ_panel, Tχ_panel, shared_interval = 1/500, tstep = 0.01);

# The Anti-Ferromagnetic Ising Model

In [29]:
# If distance is one, connection is -1
wg = @WG "dr -> dr == 1 ? -1 : 0" NN=1 
genAdj!(ig[1], wg)
simulate(ig, overwrite = true)

Simulation already active, create a new one and overwrite it? [y/n]
Character entered: y
Starting Interface


Displaying


IsingGraph with 40000 states
Layers:
Discrete IsingLayer 1 with size (200, 200) and stateset (-1.0f0, 1.0f0)
 at coordinates (0, 0, 0)
 with connections:
	Connected to layer 1 using 
 and 0 defects

	WeightGenerator with
	 NN: 				1
	 func: 				dr -> dr == 1 ? -1 : 0


# Boltzmann Machine
## How do Ising models relate to Machine Learning?

### Example: Learning statistics from MNIST
- The MNIST dataset contains images of handrwritten letters of 28x28 pixels

- These define a probability distribution over the pixels $ P_{data}(s_1,s_2,...)$

<img src="MNIST.jpg" width="750" />

___
## Boltzmann Machine: We can nudge part of the distribution of the Ising Model towards the data distribution
<img src="BM.png" width="300" />

##### Visible units $v_i$, hidden units $h_i$
##### Probability distribution over visible units $P_v(\sigma_1, \sigma_2, ...,W_{ij}) = \sum_h \frac 1 Z e^{-BH}$
##### Where $H = -\sum_{ij} W_{ij}\sigma_i\sigma_j$

#### Function that gives us the similarity of distributions: KL-Divergence
#### $KL(P^1,P^2) = \sum_{all states} P^1 \log\left( \frac {P^1}{P^2}\right)$

#### $KL(P_{data}, P_v) = \sum_{all states} P_{data} \log \left( \frac{P_{data}}{P_v}\right) = KL(W_{ij})$

- #### Gives us a scalar value that decreases when the distributions are similar, increases when they are less similar
- #### $P_{data}$ is fixed, so we can take the derivative w.r.t. the entries in the connection matrix $W_{ij}$
- #### $\frac{\partial KL}{\partial W_{ij}} = -\beta (\langle \sigma_i \sigma_j\rangle_{data} - \langle \sigma_i \sigma_j \rangle_{free}) = -\beta (p_{ij}^{data}-p_{ij}^{free})$


In [2]:
InteractiveIsing.closewindow(layerwindow)

ig = IsingGraph(architecture = [(28,28, Discrete), (32,32, Discrete)], sets = [(0,1),(0,1)], precision = Float64)
simulate(ig, overwrite = true)
set_colorrange(ig[1])

InteractiveIsing.loadparameters(ig, "rbm_data.jld2")
w2 = LayerWindow(ig[2])


Simulation already active, create a new one and overwrite it? [y/n]
Character entered: y
Starting Interface


Displaying


LayerWindow(Discrete IsingLayer 2 with size (32, 32) and stateset (0.0, 1.0)
 at coordinates (0, 0, 1)
 with connections:
 and 0 defects, Scene (800px, 600px):
  0 Plots
  1 Child Scene:
    └ Scene (800px, 600px), GLMakie.Screen(...), Dict{String, Any}(), PTimer(InteractiveIsing.var"#489#491"{InteractiveIsing.var"#484#485"{Observables.Observable{Base.ReshapedArray{Float64, 2, SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int32}}, true}, Tuple{}}}}}(InteractiveIsing.var"#484#485"{Observables.Observable{Base.ReshapedArray{Float64, 2, SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int32}}, true}, Tuple{}}}}(Observable([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0]))), Timer(Ptr{Nothing} @0x00000002f0811480, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000000010d0ed5f0, Task (runnable) @0x000000010d0ed5f0), Base.Threads.SpinLock(0)), true, false), 0.0, 0.016666666666666666))

# Visualizing the Connections

In [18]:
include("ImageGrid.jl")
creategrid(conns(ig[1],ig[2]), (28,28), (5,5))

Imgsize = (28, 28)


In [17]:
pause(ig)

1-element BitVector:
 1

In [14]:
restart(ig)